# 使用嵌入进行文档搜索 
## 概览
此示例演示了如何使用 Gemini API 创建嵌入，以便执行文档搜索。您将使用 Python 客户端库构建字词嵌入，以便将搜索字符串或问题与文档内容进行比较。
在本教程中，您将使用嵌入对一组文档执行文档搜索，以提出与 Google 汽车相关的问题。

## 设置
首先，下载并安装 Gemini API Python 库。

In [ ]:
!pip install -U -q google.generativeai

In [1]:
import textwrap
import numpy as np
import pandas as pd
import os

import google.generativeai as genai

from dotenv import load_dotenv
load_dotenv()

from IPython.display import Markdown

### 获取 API 密钥

In [2]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


## 嵌入生成
在本部分中，您将了解如何使用 Gemini API 中的嵌入为一段文本生成嵌入。

In [4]:
title = "The next generation of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications")

model = 'models/text-embedding-004'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding)

{'embedding': [-0.0021609084, -0.0031644478, -0.060120754, -0.0071218493, 0.0008775481, 0.040581927, 0.04457149, 0.03552469, -0.04746538, 0.008888608, -0.02795826, 0.011335696, -0.0024438712, 0.0030851837, -0.018796137, -0.055550937, 0.03142646, 0.0006549187, -0.11370059, 0.063708074, -0.021750016, -0.021367034, -0.09982074, -0.008604756, -0.03330058, -0.012815642, 0.07153146, 0.037064772, 0.022970116, 0.043331202, 0.010670608, 0.040685344, 0.036361404, -0.036222056, -0.01779936, -0.014820963, 0.0053205034, -0.017382707, 0.07044941, 0.002021254, -0.018208738, 0.01755808, 0.006493214, 0.12724239, -0.023805201, 0.010057808, -0.0006948924, 0.070856266, -0.05645728, 0.018311134, 0.090462275, 0.021575555, -0.06656089, 0.026865067, -0.0034812444, -0.0011228676, -0.06535635, -0.0018169138, 0.08672994, 0.028747609, -0.024817271, 0.004653876, -0.058998518, 0.032061696, -0.022604035, -0.015454275, -0.013758674, 0.021129983, -0.047893394, 0.02573244, 0.01302824, -0.018002003, -0.039879415, 0.0234

## 构建嵌入数据库
以下是用于构建嵌入数据库的三个示例文本。您将使用 Gemini API 为每个文档创建嵌入。将它们转换为 DataFrame，以获得更好的可视化效果。

In [5]:
DOCUMENT1 = {
    "title": "操作气候控制系统",
    "content": "您的Googlecar配备了一个气候控制系统，允许您调节车内的温度和气流。要操作气候控制系统，请使用位于中央控制台上的按钮和旋钮。\n温度：温度旋钮控制车内的温度。顺时针转动旋钮以增加温度，逆时针转动旋钮以降低温度。\n气流：气流旋钮控制车内的气流量。顺时针转动旋钮以增加气流，逆时针转动旋钮以减少气流。\n风扇速度：风扇速度旋钮控制风扇的速度。顺时针转动旋钮以增加风扇速度，逆时针转动旋钮以降低风扇速度。\n模式：模式按钮允许您选择所需的模式。可用的模式有：\n自动：汽车将自动调节温度和气流以保持舒适的水平。\n冷却：汽车将吹入冷空气。\n加热：汽车将吹入暖空气。\n除霜：汽车将吹入暖空气到挡风玻璃上以除霜。"
}

DOCUMENT2 = {
    "title": "触摸屏",
    "content": "您的Googlecar配备了一个大尺寸触摸屏显示器，提供了访问各种功能，包括导航、娱乐和气候控制。要使用触摸屏显示器，只需触摸所需图标。\n例如，您可以触摸“导航”图标以获取前往目的地的路线，或触摸“音乐”图标以播放您喜欢的歌曲。"
}

DOCUMENT3 = {
    "title": "换挡",
    "content": "您的Googlecar配备了自动变速器。要换挡，只需将换挡杆移动到所需位置。\n驻车：此位置用于停车时使用。车轮被锁定，汽车无法移动。\n倒车：此位置用于倒车。\n空挡：此位置用于在红灯或交通拥堵时停车。汽车不在档位中，除非您踩下油门踏板，否则不会移动。\n驾驶：此位置用于前进。\n低速：此位置用于在雪地或其他湿滑条件下驾驶。"
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [6]:
## 将字典的内容整理到 DataFrame 中，以获得更好的可视化效果。
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,操作气候控制系统,您的Googlecar配备了一个气候控制系统，允许您调节车内的温度和气流。要操作气候控制系统...
1,触摸屏,您的Googlecar配备了一个大尺寸触摸屏显示器，提供了访问各种功能，包括导航、娱乐和气候...
2,换挡,您的Googlecar配备了自动变速器。要换挡，只需将换挡杆移动到所需位置。\n驻车：此位置...


In [7]:
## 获取每个文本正文的嵌入。将此信息添加到 DataFrame 中。
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,操作气候控制系统,您的Googlecar配备了一个气候控制系统，允许您调节车内的温度和气流。要操作气候控制系统...,"[0.019978223, 0.03769091, -0.04880996, 0.02175..."
1,触摸屏,您的Googlecar配备了一个大尺寸触摸屏显示器，提供了访问各种功能，包括导航、娱乐和气候...,"[0.020754363, 0.034159817, -0.04240618, 0.0306..."
2,换挡,您的Googlecar配备了自动变速器。要换挡，只需将换挡杆移动到所需位置。\n驻车：此位置...,"[0.015702037, 0.038155437, -0.04673463, 0.0177..."


## 包含问答功能的文档搜索
现在，嵌入已生成，接下来让我们创建一个问答系统来搜索这些文档。您将提出一个有关超参数调节的问题，创建问题的嵌入，并将其与 DataFrame 中的嵌入集合进行比较。
问题的嵌入将是一个矢量（浮点值列表），将与使用点积的文档矢量进行比较。从 API 返回的这个矢量已经标准化。点积表示两个向量在方向上的相似度。
点积的值可介于 -1 和 1 之间（含 -1 和 1）。如果两个向量之间的点积为 1，那么这两个向量的方向相同。如果点积值为 0，则这些向量彼此正交或无关。最后，如果点积为 -1，则向量指向相反方向且彼此不相似。
请注意，使用新嵌入模型 (embedding-001) 时，对于用户查询，将任务类型指定为 QUERY；对于嵌入文档文本，请将任务类型指定为 DOCUMENT。

In [11]:
query = "如何在谷歌汽车中换挡？"
model = 'models/text-embedding-004'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

In [12]:
## 使用 find_best_passage 函数计算点积，然后按照从大到小的点积值对 DataFrame 进行排序，以便从数据库中检索相关段落。
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

In [13]:
## 查看数据库中最相关的文档：
passage = find_best_passage(query, df)
passage

'您的Googlecar配备了一个大尺寸触摸屏显示器，提供了访问各种功能，包括导航、娱乐和气候控制。要使用触摸屏显示器，只需触摸所需图标。\n例如，您可以触摸“导航”图标以获取前往目的地的路线，或触摸“音乐”图标以播放您喜欢的歌曲。'

## 问答应用
我们来尝试使用文本生成 API 来创建问与系统。在下面输入您自己的自定义数据，以创建简单的问答示例。您仍然会使用点积作为相似度指标。

In [ ]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""是一个有用且信息丰富的机器人，使用下面包含的参考文本来回答问题。
请务必用完整的句子回答，内容要全面，包括所有相关的背景信息。
不过，你在与非技术观众交流，所以要确保简化复杂的概念，
并保持友好和对话的语气。
如果文本与答案无关，你可以忽略它。
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [ ]:
prompt = make_prompt(query, passage)
print(prompt)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')
answer = model.generate_content(prompt)

Markdown(answer.text)